# Big DWH Week

## A. Критическая уязвимость

В n кластерах расположены серверы, на которых установлена одинаковая операционная система. В её ядре обнаружена уязвимость, поэтому необходимо обновить серверы до новой версии ядра.

В i-м кластере расположено xi серверов. Обновление одного сервера занимает одну единицу времени. Обновление серверов происходит последовательно, поэтому общее время обновления всех серверов в i-м кластере составляет xi единиц времени.

Процесс обновления в одном кластере нельзя прерывать до тех пор, пока все серверы в нём не будут обновлены. Также нельзя одновременно обновлять серверы в двух кластерах.

Для каждого кластера выделено окно времени [ai, ai + xi], в течение которого можно проводить обновление. Эти окна могут пересекаться.

Необходимо выбрать кластеры, в которых будет проведено обновление, чтобы на как можно большем количестве серверов была установлена новая версия ядра.

### Формат ввода

В первой строке дано число кластеров n.

В каждой из n следующих строк дана пара чисел ai и xi. 

### Формат вывода

В первой строке выходного файла нужно вывести максимальное суммарное количество серверов, на которых возможно обновить ядро.

Во второй строке выходного файла нужно вывести разделённые пробелом номера кластеров, в которых следует обновить ядро. Кластеры нумеруются с нуля. Номера можно вывести в произвольном порядке.


### Пример 1

```
4
1 4
4 11
8 5
12 5
```

```
11
1 
```

### Пример 2

```
4
1 4
4 11
8 3
12 5
```

```
12
3 2 0 
```


### Пример 3

```
2
1 1
2 1
```

```
2
1 0 
```

### Решение

In [1]:
def UpdateMaxServers1(n, a, x):
    
    a = a.copy()
    x = x.copy()

    inteavals = [(a[i], a[i] + x[i], x[i], i) for i in range(n)]
    inteavals = sorted(inteavals, key=lambda z: z[1])
    
    a = [inteavals[i][0] for i in range(n)]
    b = [inteavals[i][1] for i in range(n)]
    x = [inteavals[i][2] for i in range(n)]
    index = [inteavals[i][3] for i in range(n)]

    f = [0 for _ in range(n)]
    nums = [[] for _ in range(n)]
    for i in range(n):
        fj = [f[j] for j in range(i, -1, -1) if b[j] <= a[i]]
        if len(fj) > 0:
            f[i] = x[i] + max(fj)
            j = f.index(max(fj))
            nums[i] = [index[i]] + nums[j]
        else:
            f[i] = x[i] 
            nums[i] = [index[i]]
    return max(f), nums[f.index(max(f))]

In [2]:
import bisect

def UpdateMaxServers(n, intervals):
    
    if n == 1:
        return intervals[0][1] - intervals[0][0], [0]
    
    intervals = sorted(intervals, key=lambda z: z[1])
    intervals = [[intervals[i][0] for i in range(n)], # start
                 [intervals[i][1] for i in range(n)], # finish
                 [intervals[i][2] for i in range(n)]] # index
    # print(*intervals, sep='\n')

    # записываем наибольшую длинну интервалов
    dp = [[] for _ in range(n)]
    # будем сохранять индексы с наидлиннейшими интервалами 
    index = [0 for _ in range(n)]
    
    for i in range(n):

        # находим позицию последнего непересекающего интервала
        r = bisect.bisect_right(a=intervals[1], x=intervals[0][i]) 
        r -= 1 
        # если нет таких интервалов
        if r == -1: 
            count_servers = intervals[1][i] - intervals[0][i] # intervals[2][i]
            nums_clusters = [intervals[2][i]]
        else:
            count_servers = dp[index[r]][0] + intervals[1][i] - intervals[0][i]
            nums_clusters = dp[index[r]][1] + [intervals[2][i]]
        dp[i] = [count_servers, nums_clusters]
        
        if i > 0 and dp[i][0] <= dp[index[i-1]][0]:
            index[i] = index[i-1] 
        else:
            index[i] = i 
    
    maxdp = max(dp)
    return maxdp

# n = int(input())
# intervals = []
# for i in range(n):
#     a, x = map(int, input().split())
#     intervals.append([a, a+x, i])

# res = UpdateMaxServers(n, intervals)
# print(res[0])
# print(*res[1])

In [3]:
import random
from tqdm import tqdm

for _ in tqdm(range(100)):
    n = random.randint(1, 10)
    # print(n)
    a = [0] * n
    x = [0] * n
    intervals = []
    for i in range(n):
        a[i] = random.randint(1, 10)
        x[i] = random.randint(1, 10)
        intervals.append([a[i], a[i]+x[i], i])
    res1 = UpdateMaxServers1(n, a, x)
    res2 = UpdateMaxServers(n, intervals)
    # print(res2)
    # break
    if res1[0] != res2[0]: # or set(res1[1]) != set(res2[1]):
        print("!"*50)
        print('n = ', n, 'a =', a, 'x =', x, sep='\n')
        print(res1)
        print(res2)
        break

100%|██████████| 100/100 [00:00<00:00, 26874.51it/s]


# B. Ресурсы в дата-центре

Ввод
```
8
54578972 99368556
79699669 54578972
64508306 99368556
56554555 64508306
27101564 81480071
89445516 27101564
93762227 81480071
89808815 93762227
4
56554555 2
93762227 79699669
99368556 2
64508306 56554555
27101564 2
99368556 79699669
93762227 2
56554555 54578972
```

Вывод
```
1 79699669
2 64508306 56554555
0 
0 
```

В сети компании есть несколько серверов, каждый из которых имеет свой уникальный номер.

Некоторые серверы соединены между собой, образуя группы, или кластеры. Два сервера относятся к одному кластеру, если между ними есть связь.

Иногда нужно скачать файл на определённый сервер. В сети есть сервис, похожий на торрент-трекер, который может показать, на каких серверах есть нужный файл.

Проблема в том, что сервер может скачивать файлы только с серверов из своего кластера.

Напишите программу, которая будет анализировать структуру сети и определять, откуда сервер может скачать нужный файл.

### Формат ввода

Первая строка содержит целое число N — количество связей между серверами.

Следующие N строк описывают связи между серверами. 
Каждая из них содержит целые числа Ai и Bi — идентификаторы соединённых серверов. У каждого сервера свой уникальный идентификатор.

Следующая строка содержит целое число Q — количество запросов на скачивание файлов.

Следующие 2Q строк описывают запросы на скачивание файлов.

Первая строка каждой пары содержит целые числа Xi и Ki — соответственно идентификатор сервера, на который нужно скачать файл, и количество серверов, содержащих файл. Вторая строка каждой пары содержит Ki целых чисел Yij — идентификаторы серверов, содержащих файл.

### Формат вывода

Для каждого запроса в отдельной строке выведите сначала целое число Rj — количество серверов, с которых можно скачать файл. Затем выведите Rj целых чисел — идентификаторы серверов, с которых можно скачать файл.

Серверы следует выводить в том порядке, в котором они перечислены в описании соответствующего запроса во входных данных.

### Пример 1

```
8
54578972 99368556
79699669 54578972
64508306 99368556
56554555 64508306
27101564 81480071
89445516 27101564
93762227 81480071
89808815 93762227
4
56554555 2
93762227 79699669
99368556 2
64508306 56554555
27101564 2
99368556 79699669
93762227 2
56554555 54578972
```

```
1 79699669
2 64508306 56554555
0 
0 
```

### Пример 2

```
10
85619126 64616465
98159933 85619126
73978229 85619126
29978081 64616465
72404745 29978081
97698445 75243921
36815728 97698445
18360411 97698445
66445821 75243921
92142380 66445821
4
97698445 4
75243921 92142380 98159933 73978229
66445821 4
29978081 92142380 73978229 97698445
18360411 4
29978081 92142380 98159933 97698445
36815728 4
64616465 92142380 97698445 29978081
```

```
2 75243921 92142380
2 92142380 97698445
2 92142380 97698445
2 92142380 97698445
```

### Решение

In [4]:
# n = int(input())
# nodes, neighbors = [], []
# for _ in range(n):
#     node, neighbour = map(int, input().split())
#     nodes.append(node)
#     neighbors.append(neighbour)
    
# q = int(input())
# x, y = [], []
# for _ in range(q):
#     xx, k = map(int, input().split())
#     x.append(xx)
#     yi = list(map(int, input().split()))
#     y.append(yi)

In [5]:
n = 8
nodes = [54578972, 79699669, 64508306, 56554555, 27101564, 89445516, 93762227, 89808815]
neighbors = [99368556, 54578972, 99368556, 64508306, 81480071, 27101564, 81480071, 93762227]
q = 4 
# идентификатор сервера, на который нужно скачать файл
x = [56554555, 99368556, 27101564, 93762227] 
# количество серверов, содержащих файл
k = [2,2,2,2]
# идентификаторы серверов, содержащих файл
y = [[93762227, 79699669],[64508306, 56554555],[99368556, 79699669],[56554555, 54578972]] 

In [6]:
from collections import defaultdict

# dct = {}
# for i in range(n):
#     if nodes[i] not in dct:
#         dct[nodes[i]] = i
# for i in range(n):
#     if neighbors[i] not in dct:
#         dct[neighbors[i]] = i + n
# print(dct)      

graph = defaultdict(list)
for i in range(n):
    node = nodes[i] # dct[nodes[i]]
    neighbor = neighbors[i] # dct[neighbors[i]]
    graph[node].append(neighbor)
    graph[neighbor].append(node)
# for key in graph:
#     print("node =", key, "neigh =", graph[key])

# x = [dct[ind] for ind in x]
# y = [[dct[ind] for ind in ind_list] for ind_list in y]

def dfs(node, num_components):
    visited[node] = num_components
    for neighbor in graph[node]:
        if neighbor not in visited:
            dfs(neighbor, num_components)
        
visited = {}
num_components = 0
for node in graph.keys():
    if node not in visited:
        dfs(node, num_components)
        num_components += 1
# visited = dict(sorted(visited.items(), key=lambda x: x[0]))
# for key in visited:
#     print("node = ", key, "num_comp =", visited[key])

components = [[] for _ in range(num_components)]
for key in visited:
    index = visited[key]
    components[index].append(key)
print(*components, sep='\n')

# r = [[] for i in range(q)]
for i in range(q):
    r = []
    server_x = x[i]
    num_comp = visited[server_x]
    # print("server_x = ", server_x, "num_comp =", num_comp)
    for server_y in y[i]:
        if server_y in components[num_comp]:
            r.append(server_y)
    print(len(r), *r)

[54578972, 99368556, 64508306, 56554555, 79699669]
[27101564, 81480071, 93762227, 89808815, 89445516]
1 79699669
2 64508306 56554555
0
0


# C. Упорядочивание серверов

Адриана проходит стажировку в новом дата-центре Яндекса, расположенном под Владимиром. В дата-центре n серверов. Они выстроены в линию один за другим и пронумерованы от 1 до n. У каждого сервера свой вес.

Для начала Адриана хочет отсортировать первые k серверов по неубыванию веса. Из-за особенностей дата-центра разрешено менять местами только пару соседних серверов, для этого необходимо количество энергии, равное максимальному из весов этих двух серверов.

Пока что Адриана не может выбрать конкретное число k, поэтому просит вас посчитать суммарные затраты энергии на упорядочивание для всех возможных значений k от 1 до n.

### Формат ввода

В первой строке дано целое число n — количество серверов.

Во второй строке через пробел записаны n целых чисел wi — вес i-го сервера.

### Формат вывода

В единственной строке выведите через пробел n целых чисел res_i — суммарное количество энергии, необходимое для упорядочивания первых i серверов.

### Пример 1

```
3
3 1 2
```

```
0 3 6 
```

### Пример 2

```
5
1 4 3 2 5
```

```
0 0 4 11 11
```

### Решение

In [7]:
n = 3
w = [3, 1, 2]
ans = [0, 3, 6]

n = 5
w = [1, 4, 3, 2, 5]
ans = [0, 0, 4, 11, 11]

In [8]:
import bisect

def wsort(w, n):
    if n == 1:
        return 0
    res = [0] * n
    prefix = [0] * n 
    for i in range(n):
        prefix[i] = prefix[i-1] + w[i]
    
    for k in range(1, n):
        j = k
        res[k] = res[k-1]
        
        right = bisect.bisect_right(a=w[:k], x=w[k])
        if right != k:
            w.insert(right, w[k])
            w.pop(k+1)
            res[k] += prefix[k-1] - prefix[right-1] if right > 0 else prefix[k-1]
            prefix[right] = prefix[right-1] + w[right] if right > 0 else w[right]
            for i in range(right + 1, k+1):
                prefix[i] = prefix[i-1] + w[i]
    return res

n = 5
w = [1, 4, 3, 2, 5]
# ans = [0, 0, 4, 11, 11]

# n = 3
# w = [3, 1, 2]
# ans = [0, 3, 6]
print(*wsort(w, n))



0 0 4 11 11


# D. Оплата за работу